<a href="https://colab.research.google.com/github/ocisse-jems/spark_bigdata/blob/main/projet_simul_velos_libre_service/2_Cas_pratique_Analyse_fichiers_logs_cyclistes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Etude de cas : Analyse des fichiers de logs des Cyclistes**

**Objectifs** :
- charger les données
- nettoyage
- calcul de la durée pour chaque trajet
  - calcul pour chaque **timestamp** dans quel numéro de trajet se trouve le cycliste: est-il dans son 1er trajet, son 2ème trajet ?
  - une fois le n° de trajet obtenu, calcul de la date minimale et maximale liée au trajet, et donc cacul d'un délai
- faire des courbes de durée de trajets

In [1]:
# install
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet 
!apt install openjdk-8-jdk-headless &> /dev/null

     |████████████████████████████████| 281.4 MB 54 kB/s 
     |████████████████████████████████| 199 kB 64.0 MB/s 


In [2]:
# SparkSession 

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
# access to drive for getting data

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **charger la donnée**

In [39]:
path = "/content/drive/MyDrive/Spark/projet_simul_velos_libre_service/data/Cyclistes/*.csv"

all_cyclistes = spark.read.format("csv").option("header", "true").load(path, inferSchema=True)

In [40]:
all_cyclistes.count()

4868396

In [41]:
all_cyclistes.printSchema()

root
 |-- id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- sur_velo: boolean (nullable = true)
 |-- velo: string (nullable = true)
 |-- vitesse: double (nullable = true)
 |-- position: string (nullable = true)
 |-- destination_finale: string (nullable = true)



**constats**
- le fichier est indexé par un **timestamp** (colonne qui nous donne l'heure interne à la simulation)
- à chaque tour les cyclistes se deplacent d'une position et enregistrent le timestamps
- les timestamps sont repetes autant de fois qu'il y a de cyclistes


## **vérifier le nb de cycles**

- on dedoublonne la colonne **timestamp**

In [42]:
all_cyclistes.select("timestamp").drop_duplicates().count()

19396

In [43]:
all_cyclistes.show(5)

+---+-------------------+--------+-----+-------+--------------------+------------------+
| id|          timestamp|sur_velo| velo|vitesse|            position|destination_finale|
+---+-------------------+--------+-----+-------+--------------------+------------------+
|246|2018-01-01 00:01:00|   false|False|   0.02|(lon:22.62 lat:9.63)|             False|
|246|2018-01-01 00:02:00|   false|False|   0.02|(lon:22.62 lat:9.63)|             False|
|246|2018-01-01 00:03:00|   false|False|   0.02|(lon:22.62 lat:9.63)|             False|
|246|2018-01-01 00:04:00|   false|False|   0.02|(lon:22.62 lat:9.63)|             False|
|246|2018-01-01 00:05:00|   false|False|   0.02|(lon:22.62 lat:9.63)|             False|
+---+-------------------+--------+-----+-------+--------------------+------------------+
only showing top 5 rows

